In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchdiffeq import odeint

from pit.dynamics.kinematic_bicycle_f110 import KinematicBicycle_Steer
from pit.integration import Euler, RK4
from pit.data_processor.csv_loader import fetch_aligned_trajectory_data

import matplotlib.pyplot as plt
import json

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

In [ ]:
class PacejkaTireModel(nn.Module):
    def __init__(self, B=10.0, C=1.9, D=1.0, E=0.97, F=0.0, S=1.0):
        super().__init__()
        self.B_f = torch.nn.Parameter(torch.tensor(B, dtype=torch.float32))
        self.C_f = torch.nn.Parameter(torch.tensor(C, dtype=torch.float32))
        self.D_f = torch.nn.Parameter(torch.tensor(D, dtype=torch.float32))
        self.E_f = torch.nn.Parameter(torch.tensor(E, dtype=torch.float32))


    def forward(self, alpha):
        """ Calculate the lateral force of the tire

        Args:
            alpha (torch.Tensor): Shape of (B, 1) or (1)
        """
        batch_mode = True if len(alpha.shape) == 2 else False
        alpha = torch.abs(alpha)
        Fy = self.D * torch.sin(self.C * torch.atan(self.B * alpha - self.E * (self.B * alpha - torch.atan(self.B * alpha))))
        return Fy


class DynamicBicycle_Steer(nn.Module):
    """
    """

    def __init__(self, lwb, st_delay, accel_delay, params_index) -> None:
        super().__init__()
        self.lwb = torch.nn.Parameter(torch.tensor(lwb, dtype=torch.float32))
        self.steer_delay_time = torch.nn.Parameter(torch.tensor(st_delay, dtype=torch.float32))
        self.accel_delay_time = torch.nn.Parameter(torch.tensor(accel_delay, dtype=torch.float32))
        self.accel_max = torch.nn.Parameter(torch.tensor(10.0, dtype=torch.float32))
        self.accel_min = torch.nn.Parameter(torch.tensor(-10.0, dtype=torch.float32))
        self.steer_k = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.float32))
        self.steer_b = torch.nn.Parameter(torch.tensor(0.0, dtype=torch.float32))
        self.speed_k = torch.nn.Parameter(torch.tensor(0.97, dtype=torch.float32))
        # Define indexes for states and control inputs
        for key, value in params_index.items():
            setattr(self, key, value)
        # real U: steer, velocity
        # model U: steer_vel, accel.
        if params_index["tire_model"]=="pacejka":
            self.tire_model = PacejkaTireModel()

    def forward(self, states, control_inputs):
        """ Get the evaluated ODEs of the state at this point

        Args:
            states (): Shape of (B, 5) or (5)
            control_inputs (): Shape of (B, 2) or (2)
        """
        batch_mode = True if len(states.shape) == 2 else False
        diff = torch.zeros_like(states)